pytorch기반 간단한 lstm 모델입니다. 중간중간 불필요한 코드는 주석 처리 하였습니다. 감사합니다.

## 라이브러리 로딩

In [1]:

import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.preprocessing import MinMaxScaler

## 데이터 다운로드

In [4]:
!wget 'https://bit.ly/3dD5MU9'

import zipfile
with zipfile.ZipFile('3dD5MU9', 'r') as existing_zip:
    existing_zip.extractall('data')

--2021-04-28 13:01:52--  https://bit.ly/3dD5MU9
Resolving bit.ly (bit.ly)... 67.199.248.10, 67.199.248.11
Connecting to bit.ly (bit.ly)|67.199.248.10|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://drive.google.com/uc?export=download&id=1kkF00wW8v0npJ8S2nA7--eMTH3gOL03z [following]
--2021-04-28 13:01:53--  https://drive.google.com/uc?export=download&id=1kkF00wW8v0npJ8S2nA7--eMTH3gOL03z
Resolving drive.google.com (drive.google.com)... 108.177.97.102, 108.177.97.113, 108.177.97.100, ...
Connecting to drive.google.com (drive.google.com)|108.177.97.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-94-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/3pjcnj71732mln28khujm7mleb821f3q/1619614875000/00192245294648390361/*/1kkF00wW8v0npJ8S2nA7--eMTH3gOL03z?e=download [following]
--2021-04-28 13:01:56--  https://doc-14-94-docs.googleusercontent.com/docs/securesc/

### 예측해야될 데이터
---
- dangjin_floating : 당진수상태양광 발전량(KW)
- dangjin_warehouse : 당진자재창고태양광 발전량(KW)
- dangjin : 당진태양광 발전량(KW)
- ulsan : 울산태양광 발전량(KW)

In [136]:
#dangjin_fcst_data = pd.read_csv('data/dangjin_fcst_data.csv') #당진지역 발전소 동네 예보
#dangjin_obs_data = pd.read_csv('data/dangjin_obs_data.csv') # 당진지역 발전소 인근 기상 관측 자료

#ulsan_fcst_data = pd.read_csv('data/ulsan_fcst_data.csv') # 울산지역 발전소 동네 에보
#ulsan_obs_data = pd.read_csv('data/ulsan_obs_data.csv') # 울산지역 인근 기상 관측 자료

#site_info = pd.read_csv('data/site_info.csv') #발전소 정보

energy = pd.read_csv('data/energy.csv') # 발전소별 발전량

### 결측치 처리

In [137]:
#dangjin_obs_data.fillna(dangjin_obs_data.mean(),inplace = True)
#ulsan_obs_data.fillna(ulsan_obs_data.mean(),inplace = True)
energy.fillna(energy.mean(),inplace = True)

### energy의 time를 index로 지정

In [138]:
energy.head()

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2018-03-01 1:00:00,0.0,0.0,0,0
1,2018-03-01 2:00:00,0.0,0.0,0,0
2,2018-03-01 3:00:00,0.0,0.0,0,0
3,2018-03-01 4:00:00,0.0,0.0,0,0
4,2018-03-01 5:00:00,0.0,0.0,0,0


In [139]:
energy = energy.set_index('time')
energy.head()

,dangjin_floating,dangjin_warehouse,dangjin,ulsan
time,,,,
2018-03-01 1:00:00,0.0,0.0,0,0
2018-03-01 2:00:00,0.0,0.0,0,0
2018-03-01 3:00:00,0.0,0.0,0,0
2018-03-01 4:00:00,0.0,0.0,0,0
2018-03-01 5:00:00,0.0,0.0,0,0


## 모델링

In [ ]:
# 데이터 타입을 float로 변경

ulsan = energy['ulsan'].values.astype(float)
dangjin_floating = energy['dangjin_floating'].values.astype(float)
dangjin_warehouse = energy['dangjin_warehouse'].values.astype(float)
dangjin	 = energy['dangjin'].values.astype(float)


'''valid_data_size = 700
train_data = ulsan[:-valid_data_size]
valid_data = ulsan[-valid_data_size:]'''

### 하이퍼 파라미터 설정

In [121]:
# hyper parameters

learning_rate = 0.0001
sequence_length = 48
optimizer = torch.optim.Adam(dangjin_floating.parameters(), lr=learning_rate)
criterion = nn.MSELoss()
epochs = 2000

In [123]:
# 데이터를 시퀀스 형태로 변경
def make_batch(input_data, sl):
    train_x = []
    train_y = []
    L = len(input_data)
    for i in range(L-sl):
        train_seq = input_data[i:i+sl]
        train_label = input_data[i+sl:i+sl+1]
        train_x.append(train_seq)
        train_y.append(train_label)
    return train_x, train_y

In [125]:
## 모델 설계

class simple_lstm(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.input_vector = 1
        self.sequence_length = 48
        self.output_vector = 100
        self.num_layers = 4
        
        self.lstm = nn.LSTM(input_size=self.input_vector, hidden_size=self.output_vector, num_layers=self.num_layers, batch_first=True)
        self.linear = nn.Sequential(
            nn.Linear(self.output_vector, 50),
            nn.Linear(50, 30),
            nn.Linear(30, 10),
            nn.Linear(10,1)
        )
        
    def forward(self, x):
        output, _ = self.lstm(x) #(hidden, cell) 데이터는 사용하지 않음
        return self.linear(output[:,-1,:])
    
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

### dangjin_floating

In [126]:

train_x, train_y = make_batch(dangjin_floating.reshape(-1,1), sequence_length)
tensor_x = torch.Tensor(train_x)
tensor_y = torch.Tensor(train_y)


dangjin_floating = simple_lstm()
dangjin_floating = dangjin_floating.to(device)


for i in range(epochs):
    dangjin_floating.train()
    tensor_x = tensor_x.to(device)
    tensor_y = tensor_y.to(device)
    output = dangjin_floating(tensor_x)
    loss = criterion(output, tensor_y.view(-1,1))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if i%100 == 0:
        print('Epoch {}, Loss {:.5f}'.format(i, loss.item()))

x_input = np.array(energy.dangjin_floating[-48:]) #  next value based on data of last year
x_input = x_input.reshape((1, 48, 1)) 
dangjin_floating_pred = []

for i in range(672):
    
    x_input = torch.Tensor(x_input)
    x_input = x_input.to(device)
    predict = dangjin_floating(x_input).cpu().detach().numpy()
    
    new_input = predict.reshape((1,1,1))
    x_input = np.concatenate((x_input[:,-47:].cpu(), new_input), axis = 1)
    dangjin_floating_pred.append(predict[0][0])

Epoch 0, Loss 51607.18750
Epoch 100, Loss 50474.10938
Epoch 200, Loss 45387.08984
Epoch 300, Loss 38815.38281
Epoch 400, Loss 36820.07812
Epoch 500, Loss 36777.92969
Epoch 600, Loss 36701.80078
Epoch 700, Loss 11610.52637
Epoch 800, Loss 3390.51074
Epoch 900, Loss 2091.19971
Epoch 1000, Loss 1824.76257
Epoch 1100, Loss 1724.69397
Epoch 1200, Loss 1677.57312
Epoch 1300, Loss 1646.18567
Epoch 1400, Loss 1621.87109
Epoch 1500, Loss 1601.47058
Epoch 1600, Loss 1583.33533
Epoch 1700, Loss 1573.04211
Epoch 1800, Loss 1547.74414
Epoch 1900, Loss 1529.49780


### dangjin_warehouse

In [127]:
train_x, train_y = make_batch(dangjin_warehouse.reshape(-1,1), sequence_length)
tensor_x = torch.Tensor(train_x)
tensor_y = torch.Tensor(train_y)


dangjin_warehouse = simple_lstm()
dangjin_warehouse = dangjin_warehouse.to(device)

optimizer = torch.optim.Adam(dangjin_warehouse.parameters(), lr=learning_rate)
criterion = nn.MSELoss()
epochs = 2000
for i in range(epochs):
    dangjin_warehouse.train()
    tensor_x = tensor_x.to(device)
    tensor_y = tensor_y.to(device)
    output = dangjin_warehouse(tensor_x)
    loss = criterion(output, tensor_y.view(-1,1))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if i%100 == 0:
        print('Epoch {}, Loss {:.5f}'.format(i, loss.item()))

x_input = np.array(energy.dangjin_warehouse[-48:]) #  next value based on data of last year
x_input = x_input.reshape((1, 48, 1)) 
dangjin_warehouse_pred= []

for i in range(672):
    
    x_input = torch.Tensor(x_input)
    x_input = x_input.to(device)
    predict = dangjin_warehouse(x_input).cpu().detach().numpy()
    
    new_input = predict.reshape((1,1,1))
    x_input = np.concatenate((x_input[:,-47:].cpu(), new_input), axis = 1)
    dangjin_warehouse_pred.append(predict[0][0])

Epoch 0, Loss 29917.44336
Epoch 100, Loss 29111.88672
Epoch 200, Loss 25745.57617
Epoch 300, Loss 21959.69141
Epoch 400, Loss 21364.75977
Epoch 500, Loss 21360.80078
Epoch 600, Loss 10386.91602
Epoch 700, Loss 2431.11304
Epoch 800, Loss 1243.15015
Epoch 900, Loss 1057.00940
Epoch 1000, Loss 998.33832
Epoch 1100, Loss 973.09003
Epoch 1200, Loss 957.60693
Epoch 1300, Loss 947.63898
Epoch 1400, Loss 944.88098
Epoch 1500, Loss 932.76764
Epoch 1600, Loss 926.85754
Epoch 1700, Loss 921.19336
Epoch 1800, Loss 915.44659
Epoch 1900, Loss 910.47699


### dangjin

In [128]:
train_x, train_y = make_batch(dangjin.reshape(-1,1), sequence_length)
tensor_x = torch.Tensor(train_x)
tensor_y = torch.Tensor(train_y)


dangjin = simple_lstm()
dangjin = dangjin.to(device)

optimizer = torch.optim.Adam(dangjin.parameters(), lr=learning_rate)
criterion = nn.MSELoss()
epochs = 2000
for i in range(epochs):
    dangjin.train()
    tensor_x = tensor_x.to(device)
    tensor_y = tensor_y.to(device)
    output = dangjin(tensor_x)
    loss = criterion(output, tensor_y.view(-1,1))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if i%100 == 0:
        print('Epoch {}, Loss {:.5f}'.format(i, loss.item()))

x_input = np.array(energy.dangjin[-48:]) #  next value based on data of last year
x_input = x_input.reshape((1, 48, 1)) 
dangjin_pred = []

for i in range(672):
    
    x_input = torch.Tensor(x_input)
    x_input = x_input.to(device)
    predict = dangjin(x_input).cpu().detach().numpy()
    new_input = predict.reshape((1,1,1))
    x_input = np.concatenate((x_input[:,-47:].cpu(), new_input), axis = 1)
    dangjin_pred.append(predict[0][0])

Epoch 0, Loss 68106.37500
Epoch 100, Loss 67076.81250
Epoch 200, Loss 62106.85938
Epoch 300, Loss 53530.99609
Epoch 400, Loss 48857.75391
Epoch 500, Loss 48557.49609
Epoch 600, Loss 48539.19922
Epoch 700, Loss 18589.34375
Epoch 800, Loss 5169.63281
Epoch 900, Loss 2888.71973
Epoch 1000, Loss 2448.78076
Epoch 1100, Loss 2310.64258
Epoch 1200, Loss 2253.53320
Epoch 1300, Loss 2195.58960
Epoch 1400, Loss 2162.60083
Epoch 1500, Loss 2134.94263
Epoch 1600, Loss 2116.73560
Epoch 1700, Loss 2087.17432
Epoch 1800, Loss 2073.54834
Epoch 1900, Loss 2047.24402


### ulsan

In [129]:
train_x, train_y = make_batch(ulsan.reshape(-1,1), sequence_length)
tensor_x = torch.Tensor(train_x)
tensor_y = torch.Tensor(train_y)


ulsan = simple_lstm()
ulsan = ulsan.to(device)

optimizer = torch.optim.Adam(ulsan.parameters(), lr=learning_rate)
criterion = nn.MSELoss()
epochs = 2000

# 학습
for i in range(epochs):
    ulsan.train()
    tensor_x = tensor_x.to(device)
    tensor_y = tensor_y.to(device)
    output = ulsan(tensor_x)
    loss = criterion(output, tensor_y.view(-1,1))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if i%100 == 0:
        print('Epoch {}, Loss {:.5f}'.format(i, loss.item()))

        
        
# 추론        
x_input = np.array(energy.ulsan[-48:]) #  next value based on data of last year
x_input = x_input.reshape((1, 48, 1)) 
ulsan_pred = []

for i in range(672):
    
    x_input = torch.Tensor(x_input)
    x_input = x_input.to(device)
    predict = ulsan(x_input).cpu().detach().numpy()
    new_input = predict.reshape((1,1,1))
    x_input = np.concatenate((x_input[:,-47:].cpu(), new_input), axis = 1)
    ulsan_pred.append(predict[0][0])

Epoch 0, Loss 14268.14746
Epoch 100, Loss 13706.44629
Epoch 200, Loss 11392.90527
Epoch 300, Loss 10203.51172
Epoch 400, Loss 10188.09375
Epoch 500, Loss 6441.70605
Epoch 600, Loss 1093.24341
Epoch 700, Loss 554.00031
Epoch 800, Loss 481.30972
Epoch 900, Loss 458.63757
Epoch 1000, Loss 448.52103
Epoch 1100, Loss 441.45630
Epoch 1200, Loss 436.88336
Epoch 1300, Loss 436.49503
Epoch 1400, Loss 430.15289
Epoch 1500, Loss 426.38940
Epoch 1600, Loss 423.52820
Epoch 1700, Loss 420.36932
Epoch 1800, Loss 417.65854
Epoch 1900, Loss 415.62704


In [130]:
submission = pd.read_csv('data/sample_submission.csv')

In [131]:
submission.iloc[:24*28, 1] = dangjin_floating_pred
submission.iloc[:24*28, 2] = dangjin_warehouse_pred
submission.iloc[:24*28, 3] = dangjin_pred
submission.iloc[:24*28, 4] = ulsan_pred

In [132]:
submission.to_csv('submission.csv',index=False)

In [89]:
submission

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2021-02-01 01:00:00,0.130256,-0.747369,0.330018,0.008067
1,2021-02-01 02:00:00,0.273770,-1.979171,-0.008422,-0.005658
2,2021-02-01 03:00:00,0.492830,-4.252793,-0.288336,-0.097679
3,2021-02-01 04:00:00,1.000553,-6.715565,0.094921,-0.486095
4,2021-02-01 05:00:00,2.784722,-8.265849,3.015755,-1.058972
...,...,...,...,...,...
1387,2021-07-08 20:00:00,0.000000,0.000000,0.000000,0.000000
1388,2021-07-08 21:00:00,0.000000,0.000000,0.000000,0.000000
1389,2021-07-08 22:00:00,0.000000,0.000000,0.000000,0.000000
1390,2021-07-08 23:00:00,0.000000,0.000000,0.000000,0.000000
